In [20]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntProgress
from IPython.display import display
from scipy.stats import mannwhitneyu

# Append base directory
import os,sys,inspect
rootname = "neuronal-sequence-test"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

# User libraries
from src.lib.qt_wrapper import gui_fnames, gui_fpath
from src.lib.data_db import BehaviouralNeuronalDatabase
from src.lib.pandas_lib import add_list_as_row, outer_product_df

%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/git/neuronal-sequence-test
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# TODO

1. Preprocessing : Apply background subtraction
2. Alignment : Ensure intervals actually match. Ask Maria for visual test of alignment
3. Apply metrics - 1 data point per trial - pool trials - maintenance vs all
    * 1D semistatic: Mean, Variance, 1D Entropy
    * ND semistatic: Avg Correlation, ND Entropy
    * 1D dynamic: Predictive Info, AR(1) efficiency
    * ND dynamic: Predictive Info, AR(1) efficiency, Orderability coeff
    
4. Test Hypotheses:
    1. Maintenance predicts Correct/Incorrect vs other phases
    2. Maintenance predicts L/R better vs other phases
    3. Validation: Prev trial C/I or L/R predicts that of next trial
    4. Predictive aspect:
        * Semi-static maintenance activity is good predictor, fine temporal details irrelevant
        * Semi-static maintenance activity is not good, fine temporal details improve prediction

In [2]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
params['root_path_data']  = gui_fpath("Path to data files", "./")
#params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/maria'

In [3]:
dataDB = BehaviouralNeuronalDatabase(params)

In [4]:
dataDB.read_neuro_files()

IntProgress(value=0, description='Read Neuro Data:', max=4)

In [5]:
dataDB.read_behavior_files()

IntProgress(value=0, description='Read Neuro Data:', max=4)

Reading /media/alyosha/Data/TE_data/mariadata/m060/Behavior_m060_20190522.mat
Reading /media/alyosha/Data/TE_data/mariadata/m060/Behavior_m060_20190524.mat
Reading /media/alyosha/Data/TE_data/mariadata/m060/Behavior_m060_20190527.mat
Reading /media/alyosha/Data/TE_data/mariadata/m060/Behavior_m060_20190606.mat
No trials found for Trial_LWhole_Mistake skipping


In [25]:
sweepDF = outer_product_df({
    "mousename"   : ["m060"],
    "performance" : ["Correct", "Mistake"],
    "direction"   : ["L", "R"],
    "datatype"    : ["raw", "high"]
})

rezDF = sweepDF.copy()
rezDF["Maintenance"] = 0.0
rezDF["All"] = 0.0
rezDF["Significance"] = 0.0

dict_drop_none = lambda d: {k:v for k,v in d.items() if v is not None}

for idx, row in sweepDF.iterrows():
    queryDict = dict_drop_none(row.to_dict())
        
    dataAll = dataDB.get_data_from_phase(None, queryDict)
    dataMt = dataDB.get_data_from_phase("Maintenance", queryDict)

    dataAllFlat = np.vstack(dataAll).flatten()
    dataMtFlat = np.vstack(dataMt).flatten()

    muAllFlat = np.mean(dataMtFlat)
    muMtFlat = np.mean(dataAllFlat)
    
    rezDF.at[idx, "Maintenance"]  = muMtFlat
    rezDF.at[idx, "All"]          = muAllFlat
    rezDF.at[idx, "Significance"] = mannwhitneyu(dataAllFlat, dataMtFlat)[1]
    
rezDF

No behaviour found for {'mousename': 'm060', 'performance': 'Mistake', 'direction': 'L', 'datatype': 'raw'} ; skipping
No behaviour found for {'mousename': 'm060', 'performance': 'Mistake', 'direction': 'L', 'datatype': 'raw'} ; skipping
No behaviour found for {'mousename': 'm060', 'performance': 'Mistake', 'direction': 'L', 'datatype': 'high'} ; skipping
No behaviour found for {'mousename': 'm060', 'performance': 'Mistake', 'direction': 'L', 'datatype': 'high'} ; skipping


,mousename,performance,direction,datatype,Maintenance,All,Significance
0,m060,Correct,L,raw,-0.059962,0.970966,3.497313e-118
1,m060,Correct,L,high,0.950861,1.111300,9.705744e-247
2,m060,Correct,R,raw,-0.108479,0.236106,5.446718e-45
3,m060,Correct,R,high,0.959903,0.986120,8.268231e-44
4,m060,Mistake,L,raw,-0.760703,-0.676497,3.226339e-06
5,m060,Mistake,L,high,0.875655,1.000277,5.221310e-09
6,m060,Mistake,R,raw,0.490120,0.744215,5.157796e-07
7,m060,Mistake,R,high,1.050503,1.014210,2.135445e-07
